In general, there are three types of constraints:
- Constraint repair
- Penalty methods
- Feasibility-preserving operations

From an implementation standpoint, constraint repair takes the updated particles at time $t$ and applies some function that modifies the particle positions to be enforce feasibility. Penalty methods instead modify the computed objective score after the particle position update as a function of the new positions and the constraints. Finally, feasibility-preserving operations overload the algebraic operations (addition and multiplication) to guarantee feasibility (e.g., using the Aitchison geometry to enforce sum-to-one constraints). 

In [2]:
include("./src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, PSO, Objectives, Designs
using LinearAlgebra

In [3]:
function unit_hypercube_constraints(d::Int)
    I_d = Matrix{Float64}(I, d, d)
    A = vcat(I_d, -I_d)
    b = vcat(ones(d), zeros(d))
    return A, b
end

unit_hypercube_constraints (generic function with 1 method)

In [8]:
A, b = unit_hypercube_constraints(3)
experiment = Experiments.create(1, 3)
experiment = Experiments.with_linear_constraints(experiment, A, b)
context = PSO.create_context(experiment, Objectives.griewank; callback = PSO.aggregate_results(;save_world=true))
runner_state, history = PSO.optimize(context)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
6 rows, 4 cols, 12 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-6); columns 0(-4); elements 0(-12) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.0000000000e-01
HiGHS run time      :       

(Main.MetaDoE.PSO.RunnerState(Main.MetaDoE.PSO.Swarm(Main.MetaDoE.PSO.ParticleState([1.933194232306819e-8; 1.863703430350795e-9; … ; -7.831147051002956e-10; 1.0792310926679471e-8;;; -3.985041065851483e-9; -9.881763474096364e-9; … ; 1.1395533680337166e-9; 1.0390806836395482e-8;;; -3.7806977266440935e-9; 3.792717681941675e-9; … ; -3.1112181860172655e-9; -1.945031941615774e-9], [7.233628757106139e-9; 3.1135910165178506e-9; … ; 1.4935321834771138e-9; 1.0739603059230395e-8;;; -2.4608709146129023e-10; -3.6977193042987664e-9; … ; -3.964964052257649e-10; -1.8158523179847052e-9;;; -5.091910650856468e-10; -2.41430931240732e-9; … ; -1.1003355436630417e-10; 1.8154722508440268e-9]), sparse([1, 67, 81, 88, 2, 24, 40, 65, 79, 2  …  51, 98, 73, 79, 99, 11, 57, 87, 97, 100], [1, 1, 1, 1, 2, 2, 2, 2, 2, 3  …  98, 98, 99, 99, 99, 100, 100, 100, 100, 100], Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 100, 100), Main.MetaDoE.PSO.ParticleFitness([2.220446049250313e-16, 0.0, 0.0, 0.0,

In [9]:
optimizer, optimal_score = PSO.get_optimizer(runner_state)
optimal_score

0.0

In [10]:
optimizer

1×3 Matrix{Float64}:
 8.4932e-9  -7.19026e-9  -4.31948e-9

In [18]:
history[1].state

Main.MetaDoE.PSO.ParticleState([8.493200020542583e-9; -4.383177199850536e-9; … ; 4.7971285223805455e-9; -9.65397947701406e-9;;; -7.1902594184436546e-9; -8.25083595932961e-9; … ; -3.6260959084270428e-9; 1.1318493353309087e-8;;; -4.319479511664378e-9; 9.243476577482226e-9; … ; -1.6471558257083514e-10; -2.657563751744621e-9], [0.2864422264324452; 0.6813785291399367; … ; 0.45269923585035055; 0.37093407790078214;;; 0.8511260599448023; 0.7060084440920997; … ; 0.46230922687600695; 0.6281562552042737;;; 0.651028909413972; 0.9255175840017575; … ; 0.14716890123313398; 0.6722711534372335])

In [27]:
import Pkg
Pkg.add("NPZ")

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = RequestError: Resolving timed out after 30001 milliseconds while requesting https://pkg.julialang.org/registries
└ @ Pkg.Registry ~/.julia/juliaup/julia-1.11.4+0.x64.linux.gnu/share/julia/stdlib/v1.11/Pkg/src/Registry/Registry.jl:77
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/workspaces/MetaDoE/Project.toml`
  No Changes to `/workspaces/MetaDoE/Manifest.toml`


In [26]:
positions = cat([h.state.positions for h in history]...; dims=2)
positions = permutedims(positions, (2, 1, 3))

501×100×3 Array{Float64, 3}:
[:, :, 1] =
  8.4932e-9   -4.38318e-9    6.96472e-9   …   4.79713e-9   -9.65398e-9
  0.275298     0.651101      0.475637         0.476363      0.397567
  0.0          0.242765      0.242591         0.0           0.0
  8.07979     -0.358415      0.156133         0.0           0.536318
  0.109315    -5.55112e-17   0.130227         0.0           0.297123
  0.158188     0.359678      0.220716     …  -0.220431      0.409736
  0.0          0.617812      0.276435        -0.215402      0.129777
  2.35601     -0.627583      0.318414         0.0           1.30695
  0.0438864    0.0           0.0778025        0.0814083     0.140191
  0.109541     0.0           0.114147         0.146677      0.163493
  0.0          0.0760442     0.234118     …   0.130161      0.326684
  0.0          0.18732      -0.208947         0.0          -0.18048
  0.0          0.0           0.0              0.0           0.0485353
  ⋮                                       ⋱                
  2.72

In [28]:
using NPZ
npzwrite("positions.npy", positions)